In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
from scipy.ndimage.filters import gaussian_filter
import seaborn as sns

In [15]:
def processData(season,team):
    df = pd.read_csv('dataset/Allseasons.csv')
    df=df[df["season"]==season]
    df['x'] = df['xCoord']
    df['xCoord'] = df['xCoord'].abs()
    shotsPerXY = df.groupby(['xCoord','yCoord']).size().reset_index(name='nbShotsLeague')
    nbTeams = len(df['teamOfShooter'].unique())

    shotsPerXY['avgShotsLeague'] = shotsPerXY['nbShotsLeague'] / nbTeams
    shotsPerTeamPerXY = df.groupby(['teamOfShooter', 'xCoord','yCoord']).size().reset_index(name='nbShotsTeam')
    excessShots = shotsPerTeamPerXY.loc[shotsPerTeamPerXY['teamOfShooter'] == team]
    excessShots = pd.merge(excessShots, shotsPerXY, on=['xCoord', 'yCoord']) 
    excessShots['excessShots'] = excessShots['nbShotsTeam'] - excessShots['avgShotsLeague']
    return excessShots


def getTeams():
    df = pd.read_csv('dataset/Allseasons.csv')
    return df["teamOfShooter"].unique()

In [16]:
def myplot(x, y, z, s, bins=1000):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins, weights=z)
    heatmap = gaussian_filter(heatmap, sigma=s)

    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap.T, extent

def plotShotMap(season_year, team, season_games):
    fig, ax = plt.subplots(figsize=(13,8.5))

    img = plt.imread('../figures/nhl_rink.png')

    ext = [-100, 100, -42.5, 42.5]
    plt.imshow(img, zorder=0, extent=ext, aspect=1)
    # Generate some test data
    x = season_games['xCoord']
    y = season_games['yCoord']
    z = season_games['excessShots']

    sigma = 64
    img, extent = myplot(x, y, z, sigma, bins=1000)
    img = ax.imshow(img, extent=extent, origin='lower', cmap='seismic', alpha=.5)
    plt.colorbar(img, ax=ax)
    ax.set_title(f"Shot map of {team} - season {season_year}/{season_year+1}")
    

    plt.show()

In [29]:
def plotKDE(season_year, team, season_games):

    x = season_games['xCoord']
    y = season_games['yCoord']
    z = season_games['excessShots']

    fig, ax = plt.subplots(figsize=(13,8.5))
    img = plt.imread('../figures/nhl_rink.png')

    ext = [-100, 100, -42.5, 42.5]
    plt.imshow(img, zorder=0, extent=ext, aspect=1)
    sns.color_palette("coolwarm", as_cmap=True)




    #Create the heatmap
    kde = sns.kdeplot(
        data = season_games,
        x='xCoord',
        y='yCoord',
        fill=True,
        weights=z-z.min(),
        alpha=.3,
        cmap='seismic',
        cbar=True,
        cbar_kws={"use_gridspec":False, "location":"right"},
    )


    plt.xlim(-100,100)
    plt.ylim(-42.5,42.5)



    plt.show()

In [30]:
@interact(
    season_year=widgets.IntSlider(min=2016, max=2020, step=1),
    team=getTeams()
)
def show_data(season_year, team):
    season_games = processData(season_year, team)
    plotShotMap(season_year, team, season_games)
    plotKDE(season_year, team, season_games)
    display(season_games)

interactive(children=(IntSlider(value=2016, description='season_year', max=2020, min=2016), Dropdown(descripti…